# WCA Development Database - Automated Download and Import

Created by Michael George (AKA Logiqx)

Download the latest database extract from https://github.com/thewca/worldcubeassociation.org/wiki/Developer-database-export

## Import Common Libraries

Import the libraries that are used throughout this notebook

In [1]:
import os
import io

import shutil

import time

## Determine the Database Details

Connection details for MySQL / MariaDB database

Note: You will need to specify the password in $HOME/.my.cnf

In [2]:
hostname = os.environ['MYSQL_HOST']
database = 'wca_dev'
username = os.environ['MYSQL_USER']

## Download the ZIP

Save the ZIP to the local machine.

In [3]:
# Hack to force IPv4 - required on my Windows laptop for Alpine 3.13 (and newer)
import Force_IPv4

# The library urllib2 will be used for the download
import urllib.request

# Start time in fractional seconds
pc1 = time.perf_counter()

# Create file handle for the ZIP
zip_url = "https://www.worldcubeassociation.org/wst/wca-developer-database-dump.zip"
req = urllib.request.Request(zip_url, headers={'User-Agent': 'Mozilla'})
infile = urllib.request.urlopen(req, timeout = 900)

# Write the ZIP to a local file
zip_fn = os.path.basename(zip_url)
with open(zip_fn, "wb") as outfile:
    shutil.copyfileobj(infile, outfile)

# Close the URL
infile.close()

# End time in fractional seconds
pc2 = time.perf_counter()

print("Download completed in %0.2f seconds" % (pc2 - pc1))

Download completed in 45.89 seconds


## Extract the SQL

Extract the SQL script from within the ZIP file.

In [4]:
# Use the zipfile library to handle the zipfile
import zipfile

# Start time in fractional seconds
pc1 = time.perf_counter()

# Open the ZIP file
with zipfile.ZipFile(zip_fn, mode="r") as zipFile:

    # Identify the SQL script
    for filename in zipFile.namelist():
        if filename.endswith(".sql"):

            # Open the SQL script, so that it can be fixed on the fly, rather than a simple extract
            with zipFile.open(filename, mode="r") as inFile:
                with open(filename, mode="w", encoding="utf-8") as outFile:
                    for line in io.TextIOWrapper(inFile, encoding="utf-8"):

                        # Fix issue that was introduced 2024-05-17 */
                        line = line.replace('999999\\- enable the sandbox mode', '999999 - enable the sandbox mode')
                        outFile.write(line)

# End time in fractional seconds
pc2 = time.perf_counter()

print("Extract completed in %0.2f seconds" % (pc2 - pc1))

Extract completed in 9.82 seconds


## Generic SQL Function

Simple function to run a SQL script using the MySQL client

In [5]:
import subprocess

def runSqlScript(source):   
    cmd = ['mysql', '--host=%s' % hostname, '--database=%s' % database, '--user=%s' % username, '--default-character-set=utf8']

    with open(source) as infile:
        proc = subprocess.Popen(cmd, stdin = infile, stdout = subprocess.PIPE, stderr = subprocess.PIPE)
        stdout, stderr = proc.communicate()
        if proc.returncode != 0:
            raise Exception('%s returned %d: %s' % (source, proc.returncode, stderr.decode('utf-8')))

## Populate the WCA Database

Note: The actual database is expected to exist already

In [6]:
# Start time in fractional seconds
pc1 = time.perf_counter()

sqlScript = 'wca-developer-database-dump.sql'
runSqlScript(sqlScript)
os.unlink(sqlScript)

# End time in fractional seconds
pc2 = time.perf_counter()

print("Load completed in %0.2f seconds" % (pc2 - pc1))

Load completed in 354.93 seconds


## All Done!